<a href="https://colab.research.google.com/github/UnpackJungHo/XRSimulator_Osaka/blob/dataProcesiing/DataProcessing_241223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import datetime

# 기본 API URL과 인증 키 설정
base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
auth_key = "wnNFMnRfQYCzRTJ0X3GATw"

# 시작 및 종료 날짜 설정
start_date = datetime.datetime(2021, 1, 14)
end_date = datetime.datetime(2021, 6, 14)

# 결과를 저장할 리스트
all_data = []

# 월별로 데이터를 요청
current_date = start_date
while current_date < end_date:
    next_month = current_date + datetime.timedelta(days=30)  # 다음 달로 이동
    if next_month > end_date:
        next_month = end_date  # 마지막 범위 설정

    # 요청 URL 생성
    url = f"{base_url}?tm1={current_date.strftime('%Y%m%d%H%M')}&tm2={next_month.strftime('%Y%m%d%H%M')}&stn=108&authKey={auth_key}"
    print(f"Requesting data from {current_date} to {next_month}")

    # API 요청
    response = requests.get(url)

    # JSON 형식인지 확인 후 데이터 추가
    if 'application/json' in response.headers.get('Content-Type', ''):
        json_response = response.json()
        all_data.append(json_response)
    else:
        print("Response is not in JSON format. Content-Type:", response.headers.get('Content-Type'))
        print("Response content:", response.text)

    # 현재 날짜를 다음 달로 갱신
    current_date = next_month

# 결과 출력
print("모든 데이터를 성공적으로 가져왔습니다!")
print(all_data)


Requesting data from 2021-01-14 00:00:00 to 2021-02-13 00:00:00
Response is not in JSON format. Content-Type: text/plain;charset=EUC-KR
Response content: #START7777
#2345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234
# YYMMDDHHMI STN  WD   WS GST  GST  GST     PA     PS PT    PR    TA    TD    HM    PV     RN     RN     RN     RN     SD     SD     SD WC WP WW                      CA  CA   CH CT        CT  CT  CT    VS   SS    SI ST    TS    TE    TE    TE    TE  ST   WH BF IR IX
#        KST  ID  16  m/s  WD   WS   TM    hPa    hPa  -   hPa     C     C     %   hPa     mm    DAY    JUN    INT    HR3    DAY    TOT -- -- ---------------------- TOT MID  MIN -------- TOP MID LOW                  GD     C     5    10    20    30 SEA    m --  -  -
202101140000 108  29  1.8  -9 -9.0   -9 1009.2

In [5]:
import requests
import datetime
import sys

def main():
    # 기본 API URL과 인증 키 설정
    base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
    auth_key = "wnNFMnRfQYCzRTJ0X3GATw"  # 필요하면 실제 키로 교체

    # 시작 및 종료 날짜 설정
    start_date = datetime.datetime(2020, 1, 1)
    end_date = datetime.datetime(2020, 6, 15)

    # 월별로 데이터를 요청
    current_date = start_date
    all_text = ""

    while current_date < end_date:
        # 다음 달로 이동(대략 30일)
        next_month = current_date + datetime.timedelta(days=30)
        if next_month > end_date:
            next_month = end_date

        # URL 생성
        url = (
            f"{base_url}?"
            f"tm1={current_date.strftime('%Y%m%d%H%M')}&"
            f"tm2={next_month.strftime('%Y%m%d%H%M')}&"
            f"stn=108&"
            f"authKey={auth_key}"
        )
        print(f"Requesting data from {current_date} to {next_month}")

        # API 요청
        response = requests.get(url)

        # 응답이 JSON 형태인지 확인하고, 아니면 텍스트로 누적
        if 'application/json' in response.headers.get('Content-Type', ''):
            # JSON 형태면 pass
            pass
        else:
            all_text += response.text + "\n"

        current_date = next_month

    # 한 줄씩 분리
    lines = all_text.splitlines()

    # 필요한 컬럼명 설정
    target_columns = ["STN", "TA", "RN", "INT", "DAY", "WW", "WC", "WP",
                      "HR3", "TOT", "TOP", "LOW", "WH", "WD", "HM", "CT"]

    # 헤더 라인 찾기
    header_line1 = None
    header_line2 = None
    for i, line in enumerate(lines):
        if line.startswith('# YYMMDDHHMI'):
            header_line1 = line
            header_line2 = lines[i+1]
            break

    if not header_line1 or not header_line2:
        print("헤더 라인을 찾지 못했습니다.")
        sys.exit(0)

    # 헤더 라인 파싱
    header_cols1 = header_line1.strip('# ').split()
    header_cols2 = header_line2.strip('# ').split()

    # 컬럼 인덱스 매핑 초기화
    col_index_map = {}

    # 특수 컬럼들의 위치 찾기
    for i, (col1, col2) in enumerate(zip(header_cols1, header_cols2)):
        # CA TOT 찾기
        if col1 == "CA" and col2 == "TOT":
            col_index_map["TOT"] = i
        # CT TOP 찾기
        elif col1 == "CT" and col2 == "TOP":
            col_index_map["TOP"] = i
        # CT LOW 찾기
        elif col1 == "CT" and col2 == "LOW":
            col_index_map["LOW"] = i
        # RN INT 찾기
        elif col1 == "RN" and col2 == "INT":
            col_index_map["INT"] = i
        # RN DAY 찾기
        elif col1 == "RN" and col2 == "DAY":
            col_index_map["DAY"] = i
        # SD HR3 찾기
        elif col1 == "SD" and col2 == "HR3":
            col_index_map["HR3"] = i
        # SD DAY 찾기
        elif col1 == "SD" and col2 == "DAY":
            col_index_map["SD_DAY"] = i
        # SD TOT 찾기
        elif col1 == "SD" and col2 == "TOT":
            col_index_map["SD_TOT"] = i

    # 나머지 일반 컬럼들의 인덱스 찾기
    for col in ["STN", "TA", "RN", "WW", "WC", "WP", "WH", "WD", "WS", "HM", "CT"]:
        if col in header_cols1:
            col_index_map[col] = header_cols1.index(col)

    # 최종 출력할 컬럼 헤더 정의
    columns = [
        "DateTime(YYYYMMDDHHMI)", "STN", "TA", "RN", "RN_INT", "RN_DAY", "WW", "WC",
        "WP", "SD_DAY", "SD_TOT", "SD_HR3", "CA_TOT", "CT_TOP", "CT_LOW",
        "WH", "WD", "WS", "HM", "CT"
    ]

    # 각 컬럼의 너비 설정
    widths = {
        "DateTime(YYYYMMDDHHMI)": 20,
        "STN": 6,
        "TA": 8,
        "RN": 8,
        "RN_INT": 8,
        "RN_DAY": 8,
        "WW": 6,
        "WC": 6,
        "WP": 6,
        "SD_DAY": 8,
        "SD_TOT": 8,
        "SD_HR3": 8,
        "CA_TOT": 8,
        "CT_TOP": 8,
        "CT_LOW": 8,
        "WH": 8,
        "WD": 6,
        "WS": 6,
        "HM": 8,
        "CT": 8
    }

    # 헤더 출력
    header = " ".join(f"{col:^{widths[col]}}" for col in columns)

    # 헤더 출력 (공백 구분 대신 콤마 구분으로 변경)
    print(",".join(columns))

    # 실제 데이터 출력
    for line in lines:
        # 주석 라인은 제외
        if line.startswith('#'):
            continue

        parts = line.split()
        # 날짜·시간 형태(12자리)를 갖고 있는지 검사
        if len(parts) > 1 and parts[0].isdigit() and len(parts[0]) == 12:
            dt = parts[0]
            values = [dt]

            # 출력 순서
            columns_order = [
                "STN", "TA", "RN", "INT", "DAY", "WW", "WC", "WP",
                "SD_DAY", "SD_TOT", "HR3", "TOT", "TOP", "LOW",
                "WH", "WD", "WS", "HM", "CT"
            ]

            for col in columns_order:
                if col in col_index_map:
                    values.append(parts[col_index_map[col]])
                else:
                    # 없는 경우 NA로 처리
                    values.append("NA")

            # 콤마로 구분된 문자열로 출력
            print(",".join(values))

if __name__ == "__main__":
    main()

Requesting data from 2020-01-01 00:00:00 to 2020-01-31 00:00:00
Requesting data from 2020-01-31 00:00:00 to 2020-03-01 00:00:00
Requesting data from 2020-03-01 00:00:00 to 2020-03-31 00:00:00
Requesting data from 2020-03-31 00:00:00 to 2020-04-30 00:00:00
Requesting data from 2020-04-30 00:00:00 to 2020-05-30 00:00:00
Requesting data from 2020-05-30 00:00:00 to 2020-06-15 00:00:00
DateTime(YYYYMMDDHHMI),STN,TA,RN,RN_INT,RN_DAY,WW,WC,WP,SD_DAY,SD_TOT,SD_HR3,CA_TOT,CT_TOP,CT_LOW,WH,WD,WS,HM,CT
202001010000,108,-6.5,0.0,-9.0,0.0,05,70,72,-9.0,-9.0,-9.0,9,0,5,-9.0,0,0.0,38.0,Sc
202001010100,108,-5.9,-9.0,-9.0,0.0,05,70,77,-9.0,-9.0,-9.0,9,0,5,-9.0,5,1.7,40.0,Sc
202001010200,108,-5.7,-9.0,-9.0,0.0,05,70,77,-9.0,-9.0,-9.0,9,0,5,-9.0,0,0.1,42.0,Sc
202001010300,108,-5.6,0.0,-9.0,0.0,05,22,72,-9.0,-9.0,-9.0,9,0,5,-9.0,0,0.0,46.0,Sc
202001010400,108,-5.4,-9.0,-9.0,0.0,05,70,72,-9.0,-9.0,-9.0,8,0,5,-9.0,0,0.0,50.0,Sc
202001010500,108,-5.2,-9.0,-9.0,0.0,05,22,72,-9.0,-9.0,-9.0,9,0,5,-9.0,0,0.0,55.

In [15]:
import requests
import datetime
import sys
import pandas as pd

def main():
    # 기본 API URL과 인증 키 설정
    base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
    auth_key = "wnNFMnRfQYCzRTJ0X3GATw"  # 실제 키로 교체 필요

    # 시작 및 종료 날짜 설정
    start_date = datetime.datetime(2013, 1, 1)
    end_date = datetime.datetime(2014, 1, 1)

    # 월별로 데이터를 요청
    current_date = start_date
    all_text = ""

    # 데이터 저장 리스트 초기화
    data = []

    # 헤더 파싱 여부
    header_parsed = False
    required_columns = ["DateTime(YYYYMMDDHHMI)", "STN", "TA", "RN", "SD_TOT", "CA_TOT", "WD", "WS", "HM"]
    required_indices = {}

    while current_date < end_date:
        # 다음 달로 이동(대략 30일)
        next_month = current_date + datetime.timedelta(days=30)
        if next_month > end_date:
            next_month = end_date

        # URL 생성
        url = (
            f"{base_url}?"
            f"tm1={current_date.strftime('%Y%m%d%H%M')}&"
            f"tm2={next_month.strftime('%Y%m%d%H%M')}&"
            f"stn=108&"
            f"authKey={auth_key}"
        )
        print(f"Requesting data from {current_date.strftime('%Y-%m-%d')} to {next_month.strftime('%Y-%m-%d')}")

        # API 요청
        try:
            response = requests.get(url)
            response.raise_for_status()  # HTTP 오류가 발생하면 예외를 발생시킴
        except requests.exceptions.RequestException as e:
            print(f"API 요청 중 오류 발생: {e}")
            sys.exit(1)

        # 응답이 JSON 형태인지 확인하고, 아니면 텍스트로 누적
        if 'application/json' in response.headers.get('Content-Type', ''):
            # JSON 형태면 로그 출력 및 넘어감
            print("JSON 응답을 받았으나 처리되지 않습니다.")
        else:
            # 텍스트 데이터 누적
            all_text += response.text + "\n"

        current_date = next_month

    # 한 줄씩 분리
    lines = all_text.splitlines()

    # 헤더 라인 찾기 - 첫 번째 주석이 아닌, 비어있지 않은 첫 번째 라인
    header_line1 = None
    header_line2 = None
    for i, line in enumerate(lines):
        if line.startswith('# YYMMDDHHMI'):
            header_line1 = line
            if i + 1 < len(lines):
                header_line2 = lines[i + 1]
            break

    if not header_line1 or not header_line2:
        print("헤더 라인을 찾지 못했습니다.")
        sys.exit(0)

    # 헤더 라인 파싱
    header_cols1 = header_line1.strip('# ').split()
    header_cols2 = header_line2.strip('# ').split()

    # 컬럼 인덱스 매핑 초기화
    col_index_map = {}

    # 필요한 컬럼들의 위치 찾기
    for i, (col1, col2) in enumerate(zip(header_cols1, header_cols2)):
        if col1 == "YYMMDDHHMI":
            col_index_map["DateTime(YYYYMMDDHHMI)"] = i
        elif col1 == "STN":
            col_index_map["STN"] = i
        elif col1 == "TA":
            col_index_map["TA"] = i
        elif col1 == "RN" and col2 == "DAY":
            col_index_map["RN"] = i
        elif col1 == "SD" and col2 == "TOT":
            col_index_map["SD_TOT"] = i
        elif col1 == "CA" and col2 == "TOT":
            col_index_map["CA_TOT"] = i
        elif col1 == "WD":
            col_index_map["WD"] = i
        elif col1 == "WS":
            col_index_map["WS"] = i
        elif col1 == "HM":
            col_index_map["HM"] = i

    # 필수 컬럼들이 모두 매핑되었는지 확인
    missing_columns = [col for col in required_columns if col not in col_index_map]
    if missing_columns:
        print(f"필수 컬럼 {missing_columns}을(를) 찾을 수 없습니다.")
        print("현재 헤더 컬럼:", header_cols1)
        sys.exit(0)

    print("필수 컬럼 인덱스 매핑:", col_index_map)  # 디버깅 출력

    # 데이터 라인 추출 (헤더 라인 이후)
    for i, line in enumerate(lines):
        # Skip comment lines
        if line.startswith('#'):
            continue
        # Skip header line
        if i == lines.index(header_line1) or i == lines.index(header_line2):
            continue
        # Process data line
        parts = line.split()
        # 필요한 컬럼이 모두 존재하는지 확인
        if len(parts) < max(col_index_map.values()) + 1:
            continue  # 필요한 컬럼이 부족한 경우 건너뜀

        row = {}
        for col in required_columns:
            idx = col_index_map[col]
            value = parts[idx] if idx < len(parts) else "NA"
            row[col] = value

        data.append(row)

    # pandas DataFrame으로 변환
    df = pd.DataFrame(data, columns=required_columns)

    # 데이터가 비어있는지 확인
    if df.empty:
        print("추출된 데이터가 없습니다.")
        sys.exit(0)

    # CSV 파일로 저장
    csv_file = "extracted_data.csv"
    df.to_csv(csv_file, index=False, encoding='utf-8-sig')
    print(f"데이터가 '{csv_file}' 파일에 성공적으로 저장되었습니다.")

    # Excel 파일로 저장
    excel_file = "extracted_data.xlsx"
    df.to_excel(excel_file, index=False)
    print(f"데이터가 '{excel_file}' 파일에 성공적으로 저장되었습니다.")

if __name__ == "__main__":
    main()


Requesting data from 2013-01-01 to 2013-01-31
Requesting data from 2013-01-31 to 2013-03-02
Requesting data from 2013-03-02 to 2013-04-01
Requesting data from 2013-04-01 to 2013-05-01
Requesting data from 2013-05-01 to 2013-05-31
Requesting data from 2013-05-31 to 2013-06-30
Requesting data from 2013-06-30 to 2013-07-30
Requesting data from 2013-07-30 to 2013-08-29
Requesting data from 2013-08-29 to 2013-09-28
Requesting data from 2013-09-28 to 2013-10-28
Requesting data from 2013-10-28 to 2013-11-27
Requesting data from 2013-11-27 to 2013-12-27
Requesting data from 2013-12-27 to 2014-01-01
필수 컬럼 인덱스 매핑: {'DateTime(YYYYMMDDHHMI)': 0, 'STN': 1, 'WD': 2, 'WS': 3, 'TA': 11, 'HM': 13, 'RN': 16, 'SD_TOT': 21, 'CA_TOT': 25}
데이터가 'extracted_data.csv' 파일에 성공적으로 저장되었습니다.
데이터가 'extracted_data.xlsx' 파일에 성공적으로 저장되었습니다.


# 새 섹션